# 3. 2 年分積分し、最初の 1 年分をスピンアップとして捨てる。後半 1 年分を 6 時間毎に保存し ておく。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# モジュールの読み込み
import sys
sys.path.append('./module')
from utils import make_lorenz96, rk4

In [ ]:
def plot_lorenz96(result):
    result = np.array(result)

    # 摂動を加えた20番目の要素をplot
    fig1, ax1 = plt.subplots()
    ax1.plot(result[:, 19])
    ax1.set_xlabel('step')
    ax1.set_ylabel('$x_{20} $')
    plt.title('plot 20th component')

    # 1,2,3番目の要素を3次元plot
    fig2 = plt.figure()
    ax2 = fig2.gca(projection='3d')
    ax2.plot(result[:, 0], result[:, 1], result[:, 2])
    ax2.set_xlabel('$x_1$')
    ax2.set_ylabel('$x_2$')
    ax2.set_zlabel('$x_3$')
    plt.title('first three components of Lorenz96')
    
    # Hovmoller Diagram
    day = 50
    Z = result[180*20:180*20 + day*20, :]
    
    fig3, ax3 = plt.subplots(figsize=(5,5))
    ax3.grid(False)
    ax3.set_xlabel('space')
    ax3.set_ylabel('time(day)')
    ax3.set_xticks(np.arange(0,40,5))
    ax3.set_yticks(np.arange(day*20, 0, -20*5))
    ax3.set_yticklabels(np.arange(180, 180+ day, 5))
    im = ax3.imshow(Z, aspect=40/(day*20), extent=[0, 40, 0, day*20], vmax=Z.max(), vmin=Z.min())
    plt.colorbar(im)
    ax3.set_title('Hovmollor diagram')
    
    plt.show()

In [ ]:
# J: 変数の数
J = 40

# 外力
F=8
lorenz = make_lorenz96(F)

# dt: 時間刻み
# 0.05で6h
dt = 0.01

# N: 時間ステップ数
# 2年分に相当
N = 360*2*20

# 計算
result = np.zeros((N,J))
x = x0
result[0] = x[:]

for n in range(1,N):
    t = n*dt
    x = scheme(t, x, dt, lorenz)
    result[n] = x[:]
        

# スピンアップを捨てる
x_one_year = result[360*20:]

# 6時間ごとを抽出
x_by_6h = x_one_year[::5]

np.save('data/true_atr.npy', x_by_6h)

# 4. Mersenne Twister 等の性質の良い乱数生成プログラムを用いて分散 1 の正規分布乱数を生成 し、3で保存した 6 時間毎の場に足しこんで、別途保存する。これを観測データとする。

標準正規分布$N(0,1)$に従うノイズを課題３で保存した真の値にのせて観測値として保存．

In [ ]:
np.random.seed(1)
obs_atr = load_true_atr + np.random.normal(loc=0, scale=1, size=load_true_atr.shape)
np.save('data/obs_atr.npy', obs_atr)

## 真の値と観測データをプロット

In [ ]:
load_true_atr = np.laod('data/true_atr.npy')
load_obs_atr = np.load('data/obs_atr.npy')

dates = [time/4 for time in range(1440)]
fig1, ax1 = plt.subplots(figsize=(20,5))
ax1.plot(dates, load_obs_atr[:, 19], '-', label='obs', lineWidth=0.5)
ax1.plot(dates, load_true_atr[:, 19],'-', label='true', lineWidth=0.5)
ax1.set_xlabel('day')
ax1.set_ylabel('$x_{20} $')
plt.title('plot 20th component')
_ = plt.legend()